# Text2SQL via Prompt Engineering


# Text-to-SQL: Bridging the Gap Between Human Language and Databases


Text-to-SQL, also known as Natural Language to SQL (NL2SQL), is a rapidly evolving technology that translates natural, everyday language into Structured Query Language (SQL) commands. This innovative approach empowers users to interact with and retrieve data from databases simply by asking questions in plain English, eliminating the need for specialized knowledge of complex SQL syntax.

At its core, Text-to-SQL acts as an intelligent translator. It leverages the power of artificial intelligence, particularly **Natural Language Processing (NLP)** and sophisticated **AI models**, to understand the user's intent and generate the corresponding SQL query. This process allows individuals without a technical background to explore and analyze data, thereby democratizing data access within an organization.



### Retrieve data

In [ ]:
!curl "https://api.mockaroo.com/api/a3056800?count=1000&key=d9f5a790" > "Employee_Data.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 84382    0 84382    0     0   5782      0 --:--:--  0:00:14 --:--:--  6451


In [ ]:
!curl "https://api.mockaroo.com/api/3dd5d0b0?count=10&key=d9f5a790" > "Department_Data.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   447    0   447    0     0    609      0 --:--:-- --:--:-- --:--:--   609


### Setup database

In [ ]:
import sqlite3
import pandas as pd
import os

In [ ]:
# Define SQL schemas for creating tables
department_schema = """
CREATE TABLE IF NOT EXISTS department (
    department_id INT PRIMARY KEY,
    department_name VARCHAR(50),
    department_code VARCHAR(9),
    department_head VARCHAR(50)
);
"""
employee_schema = """
CREATE TABLE IF NOT EXISTS employee (
    employee_id INT PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    gender VARCHAR(50),
    date_of_birth DATE,
    email VARCHAR(50),
    phone_number VARCHAR(50),
    department_id INT,
    salary DECIMAL(8,2),
    FOREIGN KEY (department_id) REFERENCES department(department_id)
);
"""

In [ ]:
db_name = 'employees.db'
if os.path.exists(db_name):
    os.remove(db_name)
    print(f"Removed existing database '{db_name}'.")

In [ ]:
COLUMN_DATA_TYPES = {
    'department': {
        'department_id': 'int64',
        'department_name': 'object',
        'department_code': 'object',
        'department_head': 'object'
    },
    'employee': {
        'employee_id': 'int64',
        'first_name': 'object',
        'last_name': 'object',
        'gender': 'object',
        'date_of_birth': 'datetime64[ns]',
        'email': 'object',
        'phone_number': 'object',
        'department_id': 'int64',
        'salary': 'float64'
    }
}

In [ ]:
# --- Database setup ---
db_name = 'employees.db'
conn = None  # Initialize connection to None

try:
    # Establish a connection to the SQLite database
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    print(f"Database '{db_name}' created and connected successfully. ✅")

    # Create tables
    cursor.execute(department_schema)
    cursor.execute(employee_schema)
    print("Tables 'department' and 'employee'are created successfully.")

    # --- Load data from CSV files into the tables using pandas ---
    csv_to_table_map = {
        '/content/employee.csv': 'customers',
        '/contentdepartment.csv': 'products'
    }

    for csv_file, table_name in csv_to_table_map.items():
        if os.path.exists(csv_file):
            print(f"\nProcessing '{csv_file}' for table '{table_name}'...")

            # Read the CSV file into a pandas DataFrame
            df = pd.read_csv(csv_file)

            # 1. Get the expected schema for the current table
            expected_schema = COLUMN_DATA_TYPES[table_name]
            expected_cols = list(expected_schema.keys())

            # 2. Handle missing/extra columns
            # Drop columns from DataFrame that are not in the schema
            df = df[df.columns.intersection(expected_cols)]

            # Add any missing columns and fill with None (which becomes NULL in SQL)
            for col in expected_cols:
                if col not in df.columns:
                    df[col] = None

            # 3. Reorder columns to match the defined schema exactly
            df = df[expected_cols]

            # 4. Enforce data types
            for col, dtype in expected_schema.items():
                if 'datetime' in dtype:
                    # Use pd.to_datetime for date/time columns, coercing errors to NaT (Not a Time)
                    df[col] = pd.to_datetime(df[col], errors='coerce')
                else:
                    # Use astype for other columns, handling potential conversion errors
                    try:
                        df[col] = df[col].astype(dtype)
                    except (ValueError, TypeError) as e:
                        print(f"  - Warning: Could not convert column '{col}' to {dtype}. Error: {e}. Leaving as is.")


            # Use the to_sql method to insert the cleaned DataFrame
            df.to_sql(table_name, conn, if_exists='append', index=False)
            print(f"  -> Data from '{csv_file}' loaded into '{table_name}' table successfully.")
        else:
            print(f"Warning: '{csv_file}' not found. Skipping data load for '{table_name}'.")

    # Commit the changes to the database
    conn.commit()
    print("\nData committed to the database successfully. 🎉")

except sqlite3.Error as e:
    print(f"Database error: {e}")
except pd.errors.EmptyDataError as e:
    print(f"Pandas error: {e}. One of the CSV files might be empty.")
except KeyError as e:
    print(f"Schema definition error: A column is missing from the TABLE_DATA_TYPES dictionary: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
finally:
    # Close the connection if it was established
    if conn:
        conn.close()
        print("Database connection closed.")

Database 'employees.db' created and connected successfully. ✅
Tables 'department' and 'employee'are created successfully.

Data committed to the database successfully. 🎉
Database connection closed.


## Setup API key
- Create an API key from google (https://aistudio.google.com/)
- Setup API Key as Secret in Colab


#### Install Gen AI library

In [9]:
! pip install google-genai

#### Import required modules

In [10]:
from google import genai
from google.colab import userdata

In [11]:
genai_client = genai.Client(api_key=userdata.get('GOOGLE_API_KEY'))

## Prompt Engineering

In [12]:
prompt = """

###ROLE###
You are a highly skilled Text-to-SQL translator with expertise in SQL syntax, database schema interpretation, and natural language understanding. You generate syntactically correct and semantically accurate SQL queries based on user input and a given database schema.

###CONTEXT###
The user is working with a relational database named `employees`. The database includes two main tables: `employee` and `department`. The goal is to allow users to input natural language queries (in English), and have the model return equivalent SQL statements that accurately extract the requested data using the given schema.

Here is the full schema:

**Department Table**
```sql
CREATE TABLE IF NOT EXISTS department (
    department_id INT PRIMARY KEY,
    department_name VARCHAR(50),
    department_code VARCHAR(9),
    department_head VARCHAR(50)
);
```


**Employee Table**

```sql
CREATE TABLE IF NOT EXISTS employee (
    employee_id INT PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    gender VARCHAR(50),
    date_of_birth DATE,
    email VARCHAR(50),
    phone_number VARCHAR(50),
    department_id INT,
    salary DECIMAL(8,2),
    FOREIGN KEY (department_id) REFERENCES department(department_id)
);
```
###TASK###
Your task is to:

1. Read a natural language query about the employees database.
2. Interpret the user's intent based on the schema provided.
3. Generate a valid SQL SELECT query that returns the expected result.
4. Ensure correct table joins, column selection, filtering, and grouping as necessary.
5. Handle aggregate functions (e.g., COUNT, AVG, SUM) where appropriate.
6. Disambiguate user terms based on schema details (e.g., "manager" → department_head, "employee name" → first_name + last_name, etc.).


###CONSTRAINTS###

* Only return a valid SQL query as output — no explanations or extra text.
* The user is using sqllite database - respond with correct and valid sqllite syntax
* Use aliases (`AS`) for column names only when the original name is ambiguous.
* Do not create or modify tables.
* Do not assume the existence of tables or columns not provided in the schema.
* Avoid subqueries unless absolutely necessary for correctness or performance.
* Prefer readability: indent joins and clauses properly.

###EXAMPLES###

**Input:** "Show me the full names and emails of all employees in the Engineering department."
**Output:**

```sql
SELECT e.first_name, e.last_name, e.email
FROM employee e
JOIN department d ON e.department_id = d.department_id
WHERE d.department_name = 'Engineering';
```

**Input:** "What is the average salary of employees in the Marketing department?"
**Output:**

```sql
SELECT AVG(e.salary) AS average_salary
FROM employee e
JOIN department d ON e.department_id = d.department_id
WHERE d.department_name = 'Marketing';
```

**Input:** "List all department heads and their department codes."
**Output:**

```sql
SELECT department_head, department_code
FROM department;
```

**Input:** "How many employees are there in each department?"
**Output:**

```sql
SELECT d.department_name, COUNT(e.employee_id) AS employee_count
FROM department d
LEFT JOIN employee e ON d.department_id = e.department_id
GROUP BY d.department_name;
```

###OUTPUT FORMAT###
Return only the sqllite SQL query as a code block using triple backticks and the `sql` language tag, like this:

```sql
-- Your SQL query here
```
"""



In [13]:
import json
def get_sql_query_via_gemini(genai_client, prompt, user_query):

  # https://www.geeksforgeeks.org/python/formatted-string-literals-f-strings-python/
  contents = f"""
  {prompt}

  Here's the user query in english you need to work on:
  {user_query}
  """
  response = genai_client.models.generate_content(model='gemini-2.5-flash', contents=contents)
  # print(response) # uncomment this and understand at the output

  # Access the usage_metadata attribute
  usage_metadata = response.usage_metadata

  # Print the different token counts
  print(f"Input Token Count: {usage_metadata.prompt_token_count}")
  print(f"Thoughts Token Count: {response.usage_metadata.thoughts_token_count}")
  print(f"Output Token Count: {usage_metadata.candidates_token_count}")
  print(f"Total Token Count: {usage_metadata.total_token_count}")

  output = response.text.replace('```sql', '').replace('```', '')

  return output


In [14]:
import sqlite3
import pandas as pd

def execute_query(query, db_name='employees.db'):

    conn = None
    try:
        # Connect to the database
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()

        # Execute the query
        print(f"\nExecuting query on '{db_name}':\n{query}")
        cursor.execute(query)

        # Fetch all results
        results = cursor.fetchall()

        # Get column names from the cursor description
        columns = [description[0] for description in cursor.description]

        # Format results as a dataframe for easier use
        results_as_dict = [dict(zip(columns, row)) for row in results]
        results_df = pd.DataFrame(results_as_dict)

        print("Query executed successfully.")
        return results_df

    except sqlite3.Error as e:
        print(f"Database error executing query: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None
    finally:
        if conn:
            conn.close()

In [15]:
def text2sql(genai_client, prompt, user_query):
  output = get_sql_query_via_gemini(genai_client, prompt, user_query)
  results = execute_query(output)
  return results

In [16]:
text2sql(genai_client, prompt, "List all employees who earn more than 70000")

Input Token Count: 881
Thoughts Token Count: 76
Output Token Count: 21
Total Token Count: 978

Executing query on 'employees.db':

SELECT *
FROM employee
WHERE salary > 70000;

Query executed successfully.


""


In [17]:
text2sql(genai_client, prompt, "Show the names and departments of female employees")

Input Token Count: 876
Thoughts Token Count: 87
Output Token Count: 52
Total Token Count: 1015

Executing query on 'employees.db':

SELECT e.first_name, e.last_name, d.department_name
FROM employee e
JOIN department d ON e.department_id = d.department_id
WHERE e.gender = 'Female';

Query executed successfully.


""


In [18]:
text2sql(genai_client, prompt, "Show all department names and the total salary expense for each")

Input Token Count: 879
Thoughts Token Count: 93
Output Token Count: 54
Total Token Count: 1026

Executing query on 'employees.db':

SELECT d.department_name, SUM(e.salary) AS total_salary_expense
FROM department d
LEFT JOIN employee e ON d.department_id = e.department_id
GROUP BY d.department_name;

Query executed successfully.


""


In [19]:
text2sql(genai_client, prompt, "List employees whose last name starts with 'S'")

Input Token Count: 878
Thoughts Token Count: 67
Output Token Count: 25
Total Token Count: 970

Executing query on 'employees.db':

SELECT first_name, last_name
FROM employee
WHERE last_name LIKE 'S%';

Query executed successfully.


""


In [20]:
text2sql(genai_client, prompt, "Show all employees whose last name is 'Rana'")

Input Token Count: 879
Thoughts Token Count: 40
Output Token Count: 50
Total Token Count: 969

Executing query on 'employees.db':

SELECT employee_id, first_name, last_name, gender, date_of_birth, email, phone_number, department_id, salary
FROM employee
WHERE last_name = 'Rana';

Query executed successfully.


""
